# Keras

## Index
* Load data
* Preprocess data
* Create model
  * Sequential model
  * Functional model
* Evaluate

Based on [this example](https://keras.io/examples/mnist_cnn/)

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import keras
from keras.models import Model,Sequential
from keras.layers import Input, LSTM, Dense,Embedding, Dropout, Flatten,Conv2D

## Load data

In [6]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Preprocess data

According to [this](https://stackoverflow.com/questions/44747343/keras-input-explanation-input-shape-units-batch-size-dim-etc), the shape for the dense must be:
* Dense: (batch_size, optional,...,optional, input_size)
* 2D convolutional layers:(batch_size, imageside1, imageside2, channels)
* 1D convolutions and recurrent layers

In [7]:
(num_elem, img_rows, img_cols)=x_train.shape

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
#We use 1 as the last element as "keras.backend.image_data_format()" is 'channels_last'

#convert to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#normalize
x_train = x_train/np.max(x_train)
x_test = x_test/np.max(x_test)

num_classes=10 # there are 10 clases (10 digits)
# convert class vectors to binary class
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Create model
We present two modalities: Sequential and Functional.

### Sequential model

The simplest approach is to the use the sequential model. Following this approach the layers are added sequentially.

In [15]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),  activation='relu', input_shape=input_shape))
model.add(Flatten()) #Dense layer retrieves a 4D tensor. Flaten makes the output "look like" y_train
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
flatten_3 (Flatten)          (None, 21632)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                216330    
Total params: 216,650
Trainable params: 216,650
Non-trainable params: 0
_________________________________________________________________


### Functional model

An alternative approach is the functional model. Following this approach we have more freedom to create the model. More info [here](https://jovianlin.io/keras-models-sequential-vs-functional).

In [16]:
input_layer = keras.Input(shape=input_shape)

#Define conv layer and use it with the input
ConvLayer = Conv2D(32, kernel_size=(3, 3),  activation='relu', input_shape=input_shape)
out_conv = ConvLayer(input_layer)

#Define flatten layer and use it with the output of the previous layer
FlattenLayer=Flatten()
out_flatten=FlattenLayer(out_conv)

output_layer = Dense(num_classes, activation='softmax')(out_flatten)

#Create the model, by puting together the input layer and the output layer
model = keras.Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
flatten_4 (Flatten)          (None, 21632)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                216330    
Total params: 216,650
Trainable params: 216,650
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

Different loss function depending on the task. More information [here](https://www.dlology.com/blog/how-to-choose-last-layer-activation-and-loss-function)


In [21]:
batch_size = 2048
epochs = 1
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 27s 452us/step - loss: 0.3957 - accuracy: 0.8818 - val_loss: 0.3052 - val_accuracy: 0.9143


## Evaluate

In [22]:
[loss, accuracy] = model.evaluate(x_test, y_test, verbose=0)
print(loss)
print(accuracy)

0.30515169498324396
0.9143000245094299


## References

[1](https://jovianlin.io/keras-models-sequential-vs-functional/)
[2](https://elitedatascience.com/keras-tutorial-deep-learning-in-python#step-3)
[3](https://keras.io/examples/mnist_cnn/)
